In [1]:
%%capture
# Compile and import local pyrossgeo module
import os, sys
owd = os.getcwd()
os.chdir('../../')
sys.path.insert(0,'../../')
!python setup.py build_ext --inplace
os.chdir(owd)

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pyrossgeo

import pandas as pd
import json

We will generate most of the simulation configuration files based on parameters we feed into the simulation.

Warnings about RAM

Allow to have names in the node populations

Explanation about how node pop and commtuer network awas construced.

Explain what age groups we have

# Generate the configuration files

### Define model

In [99]:
model = {
    "settings" : {
        "classes" : ["S", "E", "A", "I", "R"],
        "stochastic_threshold_from_below" : [1000, 1000, 1000, 1000, 1000],
        "stochastic_threshold_from_above" : [500, 500, 500, 500, 500]
    },

    "S" : {
        "linear"    : [],
        "infection" : [ ["I", "-betaI"], ["A", "-betaA"] ]
    },

    "E" : {
        "linear"    : [ ["E", "-gammaE"] ],
        "infection" : [ ["I", "betaI"], ["A", "betaA"] ]
    },

    "A" : {
        "linear"    : [ ["E", "gammaE"], ["A", "-gammaA"] ],
        "infection" : []
    },

    "I" : {
        "linear"    : [ ["A", "gammaA"], ["I", "-gammaI"] ],
        "infection" : []
    },

    "R" : {
        "linear"    : [ ["I", "gammaI"] ],
        "infection" : []
    }
}

model_classes = model['settings']['classes']
model_dim = len(model_classes)

### Configuration generation parameters

In [91]:
sim_config_path = 'london_simulation'

min_num_moving = 100 # Remove all commuting edges where less than `min_num_moving` are moving

# Decide which classes are allowed to commute
allow_class = [
    ('S', True),
    ('E', True),
    ('A', True),
    ('Ia1', True),
    ('Ia2', True),
    ('Ia3', True),
    ('Is1', True),
    ('Is2', False),
    ('Is3', False),
    ('R', True),
]

# Decide where to seed with infecteds
seed_pop = [
    (0, 1, 'E', 10)      # Home, age group, model class, seed quantity
]

### Format the commuting network

In [92]:
cn = pd.read_csv("%s/commuter_networks.csv" % sim_config_path)

#### Set which classes are allowed to commute

# Drop the current allow_O columns
cn = cn.iloc[:,:10]

# Set allow settings
for O, allow_O in allow_class:
    cn[ "Allow %s" % O ] = 1 if allow_O else 0
    
# Allow people to return home
cn.loc[ cn['Home'] == cn['To'],"Allow %s" % allow_class[0][0]:] = 1

#### Remove commuting edges where fewer than `min_num_moving` people are commuting

delete_rows = []

for i, row in cn.loc[ cn['Home'] == cn['From'] ].iterrows():
    if row['# to move'] < min_num_moving:
        delete_rows.append(i)
        delete_rows.append(i+1) # Delete the returning commuting edge as well

cn = cn.reset_index()
cn = cn.drop(delete_rows)

cn.head()

,index,Age,Home,From,To,# to move,% to move,%,t1,t2,...,Allow S,Allow E,Allow A,Allow Ia1,Allow Ia2,Allow Ia3,Allow Is1,Allow Is2,Allow Is3,Allow R
6474,6474,1,543,543,185,112.903831,-1.0,-1,8.816575,9.000000,...,1,1,1,1,1,1,1,0,0,1
6475,6475,1,543,185,543,-1.000000,1.0,-1,17.000000,17.183425,...,1,1,1,1,1,1,1,1,1,1
8026,8026,1,939,939,185,100.729741,-1.0,-1,8.965466,9.000000,...,1,1,1,1,1,1,1,0,0,1
8027,8027,1,939,185,939,-1.000000,1.0,-1,17.000000,17.034534,...,1,1,1,1,1,1,1,1,1,1
8028,8028,2,939,939,185,106.653992,-1.0,-1,8.965466,9.000000,...,1,1,1,1,1,1,1,0,0,1


### Populate the network

Our `node_populations.csv` currently only has the total population for each age group at each node. In order to use it for the simulation, we must populate it with the model classes, as well as seed some infections.

In [101]:
tot_pop = pd.read_csv("%s/node_populations.csv" % sim_config_path)
tot_pop.head()

,Home,Location,Area,N(0-4),N(5-10),N(11-15),N(16-24),N(25-34),N(35-49),N(50-64),N(65-74),N(75+)
0,0,0,2.905399,414,601,250,910,1333,1624,1695,839,654
1,1,1,2.165634,636,904,436,765,1057,1476,1036,432,550
2,2,2,2.143565,845,1067,503,1094,1734,2261,1713,662,585
3,3,3,2.490215,441,534,265,796,930,1113,1145,432,566
4,4,4,1.186180,934,1189,562,1078,1517,2116,1404,485,397


In [102]:
# Create all model classes, and set everyone to be susceptible

np = pd.DataFrame()
np['Home'] = tot_pop['Home']
np['Location'] = tot_pop['Location']
np['Area'] = tot_pop['Area']

for cn, cd in tot_pop.iloc[:,3:].iteritems():
    for O in model['settings']['classes']:
        np["%s%s" % (O, cn[1:])] = 0
        
    np["%s%s" % ("S", cn[1:])] = cd
    
# Seed with infecteds

for home, age, O, seed_quantity in seed_pop:
    row_i = np[np['Home'] == home_node].index[0]
    col_i = 3 + age*model_dim
    S = np.iloc[row_i,col_i]
    np.iloc[row_i, col_i + model_classes.index('E')] = seed_quantity
    np.iloc[row_i, col_i] -= seed_quantity

### Setting the node and cnode parameters

### Contact matrices